In [1]:
import json
import pandas as pd
import numpy as np

from pprint import pprint

import panel as pn
import hvplot.pandas  # noqa
import holoviews as hv
from bokeh.models import HoverTool

pd.options.plotting.backend = 'holoviews'

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
players_df = pd.read_csv("data/players.csv")
clubs_df = pd.read_csv("data/clubs.csv")

full_df = pd.merge(players_df, 
                   clubs_df,
                   how='left', 
                   on='club_id',
                   suffixes=(None, '_club'))

In [3]:

# Translations and settings

with open("i18n/translations.json", "r") as f:
    translations = json.loads(f.read())
    

with open("i18n/countries_translations.json", "r") as f:
    countries_translations = json.loads(f.read())
    
field_positions_colors = {
    'GOALKEEPER' : '#fc4f30',
    'FORWARD' : '#e5ae38', 
    'DEFENDER' :  '#6d904f', 
    'MIDFIELDER' : '#30a2da',
}

lang_id = "en"

def _(x, translations_dict=None, lg_id=None):
    
    if translations_dict is None:
        translations_dict = translations
        
    if lg_id is None:
        lg_id = lang_id
        
    if x in translations_dict:
        return translations_dict[x][lg_id]
    else:
        return x

In [4]:
# HR like "Human Readable"
full_df['field_position_hr'] = full_df['field_position'].transform(lambda x:_(x))
full_df['field_position_color'] = full_df['field_position'].transform(lambda x: field_positions_colors[x] )

full_df['country_name'] = full_df['country_code'].transform(lambda x:_(x, countries_translations) )
full_df['country_flag'] = full_df['country_code'].transform(lambda x:_(x, countries_translations, 'flag') )

full_df['country_name_club'] = full_df['country_code_club'].transform(lambda x:_(x, countries_translations) )
full_df['country_flag_club'] = full_df['country_code_club'].transform(lambda x: _(x, countries_translations, 'flag') )

In [5]:
full_df.head(2)

,jersey_number,field_position,international_name,birth_date,club_id,detailed_field_position,id,image_url,country_code,field_position_fr,height,weight,team_id,age,bmi,association_logo_url,big_logo_url,country_code_club,id_provider,international_name_club,logo_url,medium_logo_url,team_code,team_type_detail,display_official_name,field_position_hr,field_position_color,country_name,country_flag,country_name_club,country_flag_club
0,1,GOALKEEPER,Salvatore Sirigu,1987-01-12,50029,GOALKEEPER,106737,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,192.0,80.0,66,34,21.701389,https://img.uefa.com/imgml/uefacom/elements/lo...,https://img.uefa.com/imgml/TP/teams/logos/700x...,ITA,FAME,Torino,https://img.uefa.com/imgml/TP/teams/logos/70x7...,https://img.uefa.com/imgml/TP/teams/logos/240x...,TOR,DOMESTIC_MEN_TEAM_A,Torino FC,Goalkeeper,#fc4f30,Italy,🇮🇹,Italy,🇮🇹
1,26,GOALKEEPER,Alex Meret,1997-03-22,50136,GOALKEEPER,250066753,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,NaN,NaN,66,24,NaN,https://img.uefa.com/imgml/uefacom/elements/lo...,https://img.uefa.com/imgml/TP/teams/logos/700x...,ITA,FAME,Napoli,https://img.uefa.com/imgml/TP/teams/logos/70x7...,https://img.uefa.com/imgml/TP/teams/logos/240x...,NAP,DOMESTIC_MEN_TEAM_A,SSC Napoli,Goalkeeper,#fc4f30,Italy,🇮🇹,Italy,🇮🇹


In [6]:
# https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/countries


with open("data/ref-countries-2020-60m.geojson/CNTR_RG_60M_2020_4326.geojson", "r") as f:
    countries = json.loads(f.read())
    
    

In [7]:
to_find = full_df.country_code.unique() 

print(len(to_find))

found = [ k['properties']['ISO3_CODE'] for k in countries['features'] if k['properties']['ISO3_CODE'] in full_df.country_code.unique() ]

['CHE', 'WAL', 'DEN', 'NED', 'CRO', 'ENG', 'SCO', 'GER', 'POR']
[c for c in to_find if c not in found]


24


['SUI', 'WAL', 'DEN', 'NED', 'CRO', 'ENG', 'SCO', 'GER', 'POR']

In [8]:

def players_height_and_weight(full_df,lang_id):
    tooltips_raw = [
        (_('dim_international_name'), '@international_name'),
        (_('dim_field_position_hr'), '@{field_position_hr}'),
        (_('dim_age'), f"@age { _('years_old') }"),
        (_('dim_country_code'), '@country_name @country_flag'),
        (_('dim_height'), '@height cm'),
        (_('dim_weight'), '@weight kg'),
        (_('dim_international_name_club'), '@international_name_club, @country_name_club @country_flag_club')
    ]


    hover = HoverTool(tooltips=tooltips_raw)

    scatter = full_df.hvplot.scatter( x='weight', 
                                      y='height', 
                                      c='field_position_color',                                   
                                      by='field_position_hr',

                                      hover_cols=[ "international_name",
                                                    "field_position_hr",
                                                    "age",
                                                    "country_name",
                                                    "country_flag",
                                                    "country_name_club",
                                                    "country_flag_club",
                                                    "weight",
                                                    "height",
                                                    "international_name_club",],
                                      height=500, 
                                      width=500, 
                                      muted_alpha=0,
                                     tools=[hover],
                                    ) \
                            .opts(
                                  legend_position='bottom_right', 
                                  legend_opts={"title":None},) \
                            .redim.label(
                                   height=_('dim_height'), 
                                   weight=_('dim_weight'), 
                                   field_position_hr= _('dim_field_position_hr'),
                                   international_name= _('dim_international_name'),
                                   age = _('dim_age'),
                                   country_code = _('dim_country_code'),
                                   international_name_club = _('dim_international_name_club'),        
                                  ) 

    shown_count = len(full_df.loc[ ~(full_df.height.isna() | full_df.weight.isna()) ])

    subtitle = _('subtitle_players_weight_and_size').replace("{%shown%}", str(shown_count)).replace("{%total%}",str(len(full_df)) )
    
    result = pn.Column(pn.pane.Markdown(f'''## {_('title_players_weight_and_size')}
_{subtitle}_'''), 
              scatter)
    
    return result

players_height_and_weight(full_df,lang_id)

Column
    [0] Markdown(str)
    [1] HoloViews(NdOverlay)

In [9]:
# We want the count of each field_position for each country
# We keep column international_name because we're sure there are no NA values for this column

def positions_distribution(full_df, lang_id):
    counts = full_df.groupby(['country_code', 'field_position']) \
                .size().reset_index(name = "count").sort_values(by="country_code", ascending=True)

    maxis = counts.groupby('field_position').max().rename(columns={'count':'maxi'})


    tooltips = f"""
    <div style="width:200px">

        <div class="bk" style="display: table; border-spacing: 2px;">
            <div class="bk" style="display: table-row;">
                <div class="bk bk-tooltip-row-label" style="display: table-cell;">{_('dim_country_code')} : </div>
                <div class="bk bk-tooltip-row-value" style="display: table-cell;">
                    <span class="bk" data-value="">@country_name</span>
                </div>
            </div>
        </div>

        <div class="bk" style="display: table; border-spacing: 2px;">
            <div class="bk" style="display: table-row;">
                <div class="bk bk-tooltip-row-label" style="display: table-cell;">{_('label_number')} : </div>
                <div class="bk bk-tooltip-row-value" style="display: table-cell;">
                    <span class="bk" data-value="">@count</span>
                </div>
            </div>
        </div>

          <div class="bk" style="display: table; border-spacing: 2px;">
            <div class="bk" style="display: table-row;">
                <div class="bk bk-tooltip-row-label" style="display: table-cell;">{_('label_players')}</div>
                <div class="bk bk-tooltip-row-value" style="display: table-cell;">
                    <span class="bk" data-value="">@international_name</span>
                </div>
            </div>
        </div>

    </div>
    """


    hover = HoverTool(tooltips=tooltips)

    plots_per_position = {}
    final_plot = None

    positions = full_df.field_position.unique()
    for p in positions:

        max_for_p = maxis.loc[p]['maxi']

        count_serie = full_df[ full_df.field_position == p ].groupby(['country_code', 'field_position']) \
                          .size().reset_index(name = "count") \
                          .set_index(['country_code', 'field_position'])

        names_serie = full_df[ full_df.field_position == p ].groupby(['country_code', 'field_position'])['international_name'].apply(lambda x: ', '.join(x))

        df_for_p = pd.concat([count_serie, names_serie], axis=1).reset_index()

        df_for_p['country_name'] = df_for_p['country_code'].transform(lambda x:"%s %s"%(countries_translations[x][lang_id],countries_translations[x]['flag'])  )

        df_for_p = df_for_p.set_index(['country_name', 'field_position']).sort_values(by="country_name", ascending=False)

        plot = df_for_p.hvplot \
            .barh(stacked=True,
                  cmap=field_positions_colors, 
                  height=450, 
                  width= max(150, int(max_for_p*20) ) + (100 if p == positions[0] else 0),
                  hover_cols=[  'country_name', 
                                'field_position_hr',
                                'count',
                                'international_name', ],
                  tools=[hover]
                 ) \
            .opts(title=_(p+"_plural"), 
                  show_legend=False,
                  xticks=[ i for i in range(max_for_p)], 
                  labelled=[]
                 )

        plots_per_position[p] = plot

        if final_plot is None:
            final_plot = plot
        else:
            final_plot += plot.opts(yaxis=None, )


    final_plot = final_plot.opts(shared_axes=False, toolbar=None)

    result = pn.Column(pn.pane.Markdown(f'''## {_('title_positions_distribution')}'''), 
                  final_plot)
    return result
    
    
positions_distribution(full_df, lang_id)

Column
    [0] Markdown(str)
    [1] HoloViews(Layout)

In [10]:
youngest = full_df[ players_df.birth_date == players_df.birth_date.max() ].to_dict('records')[0]
oldest = full_df[ players_df.birth_date == players_df.birth_date.min()  ].to_dict('records')[0]

tallest = full_df[ players_df.height == players_df.height.max() ].to_dict('records')
smallest = full_df[ players_df.height == players_df.height.min()  ].to_dict('records')[0]


pn.pane.Markdown(f'''**Joueur le plus jeune :**  {youngest['international_name']} {youngest['country_flag']}  né le {youngest['birth_date']} ({youngest['age']} ans)  
**Joueur le plus vieux :**  {oldest['international_name']} {oldest['country_flag']}  né le {oldest['birth_date']} ({oldest['age']} ans)  
**Joueurs les plus grand :**  
    - {tallest[0]['international_name']} {tallest[0]['country_flag']}  {tallest[0]['height']} cm,  {tallest[0]['field_position_hr']}   
    - {tallest[1]['international_name']} {tallest[1]['country_flag']}  {tallest[1]['height']} cm,  {tallest[1]['field_position_hr']}   
**Joueur le plus petit :**  {smallest['international_name']} {smallest['country_flag']}  {smallest['height']} cm, {smallest['field_position_hr']}  
''', width=800)

Markdown(str, width=800)

In [11]:
    
mean_ages_df = pd.DataFrame(full_df.groupby("country_code") \
            .mean()['age']\
            .sort_values(ascending=True)).reset_index()
mean_ages_df['country_name'] = mean_ages_df['country_code'] \
                                .transform(lambda x:"%s %s"%(countries_translations[x][lang_id],countries_translations[x]['flag'])  )

mean_ages_df = mean_ages_df.set_index('country_name')


mean_ages_df.hvplot.barh(height=600) \
                    .opts(labelled=[], 
                          title="Age moyen des joueurs par équipe") \
                    .redim.label(age='Age moyen', country_name='Equipe')

:Bars   [country_name]   (age)

In [12]:
total_counts = full_df.groupby(["country_code"]).size().to_dict()

# Which national teams rely the most on their local leagues?

#count_per_country_club = full_df.groupby(["country_code", "country_code_club"]).count()['jersey_number'].reset_index().rename(columns={"jersey_number":"count"})
count_per_country_club = full_df.groupby(["country_code", "country_code_club"]).size().reset_index(name="count")

count_per_country_club = count_per_country_club.loc[  count_per_country_club['country_code'] == count_per_country_club['country_code_club'] , ["country_code", "count"] ].sort_values(by="count", ascending=True)

count_per_country_club['percentage'] = count_per_country_club.apply( lambda x: round(x['count'] / total_counts[x['country_code']] *100,1) , axis=1 )


count_per_country_club['country_name'] = count_per_country_club['country_code'] \
                                .transform(lambda x:"%s %s"%(countries_translations[x][lang_id],countries_translations[x]['flag'])  )

count_per_country_club = count_per_country_club.set_index('country_name')


chart = count_per_country_club.hvplot.barh("country_name", "count", height=600, color='count', cmap='kgy') \
                    .opts(labelled=[], 
                          title="Quelles équipes ont selectionné des joueurs de leurs propres ligues ?") \
                    .redim.label(count='Nombres de joueurs sélectionnés', country_name='Equipe')

chart

:Bars   [country_name]   (count)

In [13]:
# Todo : add the total number of countries 

#count_per_country_club = full_df.groupby(["country_code", "country_code_club"]).count()['jersey_number'].reset_index().rename(columns={"jersey_number":"count"})
count_per_country_club = full_df.groupby(["country_code", "country_code_club"]).size().reset_index(name = "count")

count_per_country_club['country_name'] = count_per_country_club['country_code'] \
                                .transform(lambda x:"%s %s"%(countries_translations[x][lang_id],countries_translations[x]['flag'])  )
count_per_country_club['country_name_club'] = count_per_country_club['country_code_club'] \
                                .transform(lambda x:"%s %s"%(countries_translations[x][lang_id],countries_translations[x]['flag'])  )
count_per_country_club['country_flag'] = count_per_country_club['country_code'] \
                                .transform(lambda x: countries_translations[x]['flag'])  



count_per_country_club =count_per_country_club \
                                .sort_values(by="country_name_club", ascending=False) \
                                .sort_values(by="country_name", ascending=False)


main_heatmap = count_per_country_club.hvplot.heatmap(y='country_name', 
                                      x='country_name_club', 
                                      C='count', 
                                      hover_cols=['count'],
                                      height=600,
                                      width=1100, 
                                      colorbar=False,
                                     cmap='kgy', # 'kbc'
                                    title='Répartition des ligues des joueurs'
                                     ) \
                                .opts(xrotation=45) \
                                .redim.label(country_name="Equipes", 
                                             country_name_club="Ligue des joueurs selectionnés",
                                            count='Total')


In [14]:
country_club_count = count_per_country_club.groupby(["country_name", "country_flag"])  \
                                            .size()   \
                                            .reset_index(name="count")  \
                                            .sort_values(by="country_name", ascending=False)

country_club_count['x'] = 1

tooltips_raw = [
    ('Pays', '@country_name'),
    ('Nombre de ligues différentes', '@count'),
]


hover = HoverTool(tooltips=tooltips_raw)

ccc_hm = country_club_count.hvplot.heatmap(y='country_flag', 
                                           x='x', 
                                           C='count', 
                                           hover_cols=['count', 'country_name'],
                                           height=500,
                                           width=150, 
                                           colorbar=True,
                                           cmap='kgy',
                                ) \
                                .opts(  tools=[hover],
                                        xaxis=None, 
                                        colorbar_position='right',
                                        yaxis='left', 
                                        ylabel='Nombre total de ligues différents'
                                ) 

main_heatmap + ccc_hm

:Layout
   .HeatMap.I  :HeatMap   [country_name_club,country_name]   (count,count)
   .HeatMap.II :HeatMap   [x,country_flag]   (count,count,country_name)

In [15]:
#count_per_club = full_df.groupby(["country_code", "international_name_club", "country_code_club"]).count()["jersey_number"].reset_index().rename(columns={"jersey_number":"count"})
count_per_club = full_df.groupby(["country_code", "international_name_club", "country_code_club"]).size().reset_index(name="count")

count_per_club['country_name'] = count_per_club['country_code'] \
                                .transform(lambda x:"%s %s"%(countries_translations[x][lang_id],countries_translations[x]['flag'])  )
count_per_club['country_name_club'] = count_per_club['country_code_club'] \
                                .transform(lambda x:"%s %s"%(countries_translations[x][lang_id],countries_translations[x]['flag'])  )

count_per_club =count_per_club \
                                .sort_values(by="country_name_club", ascending=False) \
                                .sort_values(by="country_name", ascending=False)

#count_per_club[ count_per_club['count'] > 3 ]

In [16]:

heatmap = count_per_club[ count_per_club['count'] > 2 ].hvplot.heatmap(
                                      x='international_name_club', 
                                      y='country_name', 
                                      C='count', 
                                      hover_cols=['count'],
                                      height=500,
                                      width=800, 
                                      colorbar=True,
                                     cmap='kgy', # 'kbc'
                                    title='Répartition des clubs des joueurs (à partir de 3 joueurs d\'un même club par équipe)'
                                     ) \
                                .opts(xrotation=45, clim=(0,10)) \
                                .redim.label(country_name="Equipes", 
                                             international_name_club="Clubs des joueurs selectionnés", 
                                            count='Total')


heatmap

:HeatMap   [international_name_club,country_name]   (count,count)

In [17]:
# First, build the data where :
# - source is the club's country (country_code_club) 
# - destination is the club's name (display_official_name)
# - volume is the number of players

sankey_base = full_df.groupby(["country_code", "country_code_club", "display_official_name","international_name"]).size().reset_index(name = "count")

sankey_base['country_country_club_junction'] = sankey_base['country_code'] + "_" + sankey_base['country_code_club']

# left part, national teams -> clubs' countries
left_part_groupby = sankey_base.groupby(["country_code", "country_country_club_junction"])
left_part = left_part_groupby["count"].sum().reset_index(name="volume")
left_part = left_part.rename(columns={
    "country_code":"source",
    "country_country_club_junction":"destination"
})

names_serie = left_part_groupby['international_name'].apply(lambda x: ', '.join(x))
names_serie = names_serie.reset_index(name='players').rename(columns={
    "country_code":"source",
    "country_country_club_junction":"destination"
})

left_part = pd.merge(left_part, 
         names_serie,
         how='outer', 
         on=['source', 'destination'])



# right part, clubs' countries -> clubs
right_part_groupby = sankey_base.groupby([ "country_country_club_junction", "display_official_name"])
right_part = right_part_groupby["count"].sum().reset_index(name="volume")

right_part = right_part.rename(columns={
    "country_country_club_junction":"source",
    "display_official_name":"destination"
})

names_serie = right_part_groupby['international_name'].apply(lambda x: ', '.join(x))
names_serie = names_serie.reset_index(name='players').rename(columns={
    "country_country_club_junction":"source",
    "display_official_name":"destination"
})

right_part = pd.merge(right_part, 
         names_serie,
         how='outer', 
         on=['source', 'destination'])


sankey_full = pd.concat([left_part, right_part])


In [18]:
#sankey_full.head(), sankey_full.tail()

In [19]:
# https://chase-seibert.github.io/blog/2011/07/29/python-calculate-lighterdarker-rgb-colors.html
def color_variant(hex_color, brightness_offset=1):
    """ takes a color like #87c95f and produces a lighter or darker variant """
    if len(hex_color) != 7:
        raise Exception("Passed %s into color_variant(), needs to be in #87c95f format." % hex_color)
    rgb_hex = [hex_color[x:x+2] for x in [1, 3, 5]]
    new_rgb_int = [int(hex_value, 16) + brightness_offset for hex_value in rgb_hex]
    new_rgb_int = [min([255, max([0, i])]) for i in new_rgb_int] # make sure new values are between 0 and 255
    # hex() produces "0x88", we want just "88"
    return "#" + "".join([hex(i)[2:] for i in new_rgb_int])

In [20]:
#sorted((full_df.country_name + ' ' +full_df.country_flag).unique())

options_raw = list(full_df.groupby(['country_name', 'country_flag', 'country_code']).groups.keys())
options = { f"{i[0]} {i[1]}":i[2] for i in options_raw }
options

{'Austria 🇦🇹': 'AUT',
 'Belgium 🇧🇪': 'BEL',
 'Croatia 🇭🇷': 'CRO',
 'Czech Republic 🇨🇿': 'CZE',
 'Denmark 🇩🇰': 'DEN',
 'England 🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f': 'ENG',
 'Finland 🇫🇮': 'FIN',
 'France 🇫🇷': 'FRA',
 'Germany 🇩🇪': 'GER',
 'Hungary 🇭🇺': 'HUN',
 'Italy 🇮🇹': 'ITA',
 'Macedonia 🇲🇰': 'MKD',
 'Netherlands 🇳🇱': 'NED',
 'Poland 🇵🇱': 'POL',
 'Portugal 🇵🇹': 'POR',
 'Russia 🇷🇺': 'RUS',
 'Scotland 🏴\U000e0067\U000e0062\U000e0073\U000e0063\U000e0074\U000e007f': 'SCO',
 'Slovakia 🇸🇰': 'SVK',
 'Spain 🇪🇸': 'ESP',
 'Sweden 🇸🇪': 'SWE',
 'Switzerland 🇨🇭': 'SUI',
 'Turkey 🇹🇷': 'TUR',
 'Ukraine 🇺🇦': 'UKR',
 'Wales 🏴\U000e0067\U000e0062\U000e0077\U000e006c\U000e0073\U000e007f': 'WAL'}

In [21]:

def sankey_for_country_code(country_code, sankey_full):

    sankey_fr = pd.DataFrame(sankey_full[ (sankey_full['source']==country_code) | (sankey_full['source'].str.startswith(f'{country_code}_')) ])

    # Build a list of colors to give as colormap for the Sankey, to get around a bug 
    # that prevents using a column as color dimension
    if len(sankey_fr['source'].unique()) > len(hv.Cycle.default_cycles['default_colors']):
        base_cmap = hv.Cycle.default_cycles['default_colors'] * 2
    else:
        base_cmap = hv.Cycle.default_cycles['default_colors']

    colormap =  list(base_cmap[ : len(sankey_fr["source"].unique()) ])
    color_dict = dict(zip(sankey_fr['source'].unique()[1:], colormap[1:]))


    for node in sankey_fr['source'].unique()[1:]:
        nbr_variations = len(sankey_fr[ sankey_fr['source'] == node ])

        # The more variations in color,
        # the smaller the offset in each variation
        variation_offset = int(30 / (nbr_variations//5 +1)) 
        for i in range(nbr_variations):
            #print(nbr_variations, variation_offset, i*variation_offset)
            colormap.append( color_variant( color_dict[node], i*variation_offset ) )


    # if country_code is FRA : 
    # - replace FRA with "France 🇫🇷"
    # - replace FRA_* with the corresponding country + flag
    # _ for FRA_FRA, replaces it with "France  🇫🇷" WITH TWO SPACES to avoid cycling graph (Sankey doesn't support them)

    sankey_fr = sankey_fr.replace(country_code, "%s %s"%(countries_translations[country_code][lang_id], countries_translations[country_code]['flag'])  )

    for node in sankey_fr['source'].unique()[1:]:

        country = node.split("_")[1]    
        if node == '%s_%s'%(country_code, country_code):
            # two spaces, to make it 
            replacement = "%s  %s"%(countries_translations[country][lang_id], countries_translations[country]['flag'])
        else:
             replacement = "%s %s"%(countries_translations[country][lang_id], countries_translations[country]['flag'])

        sankey_fr = sankey_fr.replace(node, replacement  )

    #sankey_fr    

    title = f"{countries_translations[country_code][lang_id]} {countries_translations[country_code]['flag']} - Clubs d'origine des joueurs"
    
    sankey = hv.Sankey(sankey_fr,
              vdims=[hv.Dimension("volume"), hv.Dimension("players")], 
              #label=title
                      ).opts(width=800, 
                                                           height=600,
                                                           edge_color='#f2f2f2',
                                                          cmap= colormap, 
                                                          label_position='right',
                                                          node_sort=False
                                                         ).redim(source='De', 
                                                                 destination="Vers", 
                                                                 volume="Nombre de joueurs", 
                                                                players='Joueurs')

    return sankey

#sankey_for_country_code('ITA')

select = pn.widgets.Select(name='Pick a team', 
                  options=options,
                  value='FRA'
                 )

bound_fn = pn.bind(sankey_for_country_code,
                   country_code=select, 
                   sankey_full=sankey_full)

pn.Column(
    pn.pane.Markdown("## Clubs d\'origine des joueurs"),
    select, bound_fn)


Column
    [0] Markdown(str)
    [1] Select(name='Pick a team', options={'Austria 🇦🇹': 'AUT', ...}, value='FRA')
    [2] ParamFunction(function)